In [6]:
%load_ext autoreload
%matplotlib widget
%autoreload 2
from amftrack.pipeline.launching.run_super import (
    run_launcher,
    directory_scratch,
    directory_project,
    directory_project,
    run_parallel_stitch,
    run_parallel_transfer,
    run_parallel_post,
)
import os
from amftrack.util.sys import (
    get_dates_datetime,
    get_dirname,
    temp_path,
    get_data_info,
    update_plate_info,
    update_analysis_info,
    get_analysis_info,
    get_current_folders,
    get_folders_by_plate_id,
)
from time import time_ns
from amftrack.util.dbx import upload_folders, load_dbx, download, get_dropbox_folders
from datetime import datetime
from amftrack.pipeline.functions.image_processing.experiment_util import (
    plot_full_image_with_features,
)
from amftrack.pipeline.functions.image_processing.experiment_class_surf import (
    load_graphs,
)
from amftrack.pipeline.functions.post_processing.extract_study_zone import (
    load_study_zone,
)
from amftrack.pipeline.functions.image_processing.experiment_util import (
    get_random_edge,
    distance_point_edge,
    plot_edge,
    plot_edge_cropped,
    find_nearest_edge,
    get_edge_from_node_labels,
    plot_full_image_with_features,
    get_all_edges,
    get_all_nodes,
    find_neighboring_edges,
    reconstruct_image,
    reconstruct_skeletton_from_edges,
    reconstruct_skeletton_unicolor,
    plot_edge_width,
    reconstruct_image_from_general,
    plot_full,
)
from amftrack.pipeline.functions.image_processing.extract_width_fun import (
    get_width_info,
    get_width_info_new,
)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [27]:
plates = ['1048_20220506','32_20220428', '33_20220502', '52_20220707','84_20220621', '91_20220708', '938_20220325']

In [12]:
directory_targ = directory_project

In [25]:
update_analysis_info(directory_targ)
analysis_info = get_analysis_info(directory_targ)
analysis_folders = analysis_info.loc[analysis_info["unique_id"].astype(str).isin(plates)]

In [26]:
analysis_folders

,folder_analysis,version,total_path,Plate,strain,medium,CrossDate,date,unique_id,datetime,...,path_time_hypha_info,path_time_plate_info,path_global_plate_info,date_run_analysis,PrincePos,root,split,Pbait,Temperature,Temp
14,Analysis_33_20220502_188_Version1,Version1,/projects/0/einf914/data/20220502_2111_Plate07,33,'Agg','001P100N100C',20220502,"02.05.2022, 21:11:",33_20220502,2022-05-02 21:11:00,...,Analysis_33_20220502_188_Version1/time_hypha_info,Analysis_33_20220502_188_Version1/time_plate_i...,Analysis_33_20220502_188_Version1/global_plate...,"01.01.1970, 01:00:",7,'Carrot','Y','-',25,NaN
15,Analysis_91_20220708_200_Version1,Version1,/scratch-shared/amftrack/stitch_temp2/20220708...,91,'A5sp3','001P100N100C',20220708,"08.07.2022, 13:53:",91_20220708,2022-07-08 13:53:00,...,Analysis_91_20220708_200_Version1/time_hypha_info,Analysis_91_20220708_200_Version1/time_plate_i...,Analysis_91_20220708_200_Version1/global_plate...,"01.01.1970, 01:00:",9,'Carrot','Y','-',25,NaN
16,Analysis_938_20220325_78_Version1,Version1,/projects/0/einf914/data/20220325_2142_Plate15,938,'A5','001P100N200C',20220325,"25.03.2022, 21:42:",938_20220325,2022-03-25 21:42:00,...,Analysis_938_20220325_78_Version1/time_hypha_info,Analysis_938_20220325_78_Version1/time_plate_i...,Analysis_938_20220325_78_Version1/global_plate...,"01.01.1970, 01:00:",15,'Carrot','Y','-',25,NaN
28,Analysis_32_20220428_200_Version1,Version1,/projects/0/einf914/data/20220428_1924_Plate20,32,'Agg','001P100N100C',20220428,"28.04.2022, 19:24:",32_20220428,2022-04-28 19:24:00,...,Analysis_32_20220428_200_Version1/time_hypha_info,Analysis_32_20220428_200_Version1/time_plate_i...,Analysis_32_20220428_200_Version1/global_plate...,"01.01.1970, 01:00:",20,'Carrot','Y','-',25,NaN
45,Analysis_1048_20220506_200_Version1,Version1,/projects/0/einf914/data/20220506_2350_Plate03,1048,'C2','001P100N100C',20220506,"06.05.2022, 23:50:",1048_20220506,2022-05-06 23:50:00,...,Analysis_1048_20220506_200_Version1/time_hypha...,Analysis_1048_20220506_200_Version1/time_plate...,Analysis_1048_20220506_200_Version1/global_pla...,"01.01.1970, 01:00:",3,'Carrot','Y','-',25,NaN
50,Analysis_52_20220707_200_Version1,Version1,/scratch-shared/amftrack/stitch_temp2/20220707...,52,'A5','001P100N100C',20220707,"07.07.2022, 14:58:",52_20220707,2022-07-07 14:58:00,...,Analysis_52_20220707_200_Version1/time_hypha_info,Analysis_52_20220707_200_Version1/time_plate_i...,Analysis_52_20220707_200_Version1/global_plate...,"01.01.1970, 01:00:",7,'Carrot','Y','-',25,nan
80,Analysis_84_20220621_200_Version1,Version1,/projects/0/einf914/data/20220621_1521_Plate01,84,'A5sp3','001P100N100C',20220621,"21.06.2022, 15:21:",84_20220621,2022-06-21 15:21:00,...,Analysis_84_20220621_200_Version1/time_hypha_info,Analysis_84_20220621_200_Version1/time_plate_i...,Analysis_84_20220621_200_Version1/global_plate...,"01.01.1970, 01:00:",1,'Carrot','Y','-',25,NaN


In [20]:
update_plate_info(directory_targ, local=True)
all_folders = get_current_folders(directory_targ, local=True)

analysed:   0%|          | 0/2250 [00:00<?, ?it/s]

In [21]:
# plates = ["1048_20220506"]
folders = all_folders.loc[all_folders["unique_id"].isin(plates)]

In [7]:
analysis_folders = analysis_info.loc[analysis_info["unique_id"].isin(plates)]

In [22]:
folders

,total_path,Plate,strain,medium,CrossDate,/Img/TileConfiguration.txt.registered,/Analysis/skeleton_compressed.mat,/Analysis/skeleton_masked_compressed.mat,/Analysis/skeleton_pruned_compressed.mat,/Analysis/transform.mat,...,/Analysis/nx_graph_pruned_labeled.p,date,folder,PrincePos,root,split,Pbait,Temperature,unique_id,datetime
1,/projects/0/einf914/data/20220326_1339_Plate15,938,'A5','001P100N200C',20220325,True,False,False,False,False,...,False,"26.03.2022, 13:39:",20220326_1339_Plate15,15,'Carrot','Y','-',25,938_20220325,2022-03-26 13:39:00
3,/projects/0/einf914/data/20220405_2204_Plate15,938,'A5','001P100N200C',20220325,True,True,True,True,True,...,True,"05.04.2022, 22:04:",20220405_2204_Plate15,15,'Carrot','Y','-',25,938_20220325,2022-04-05 22:04:00
4,/projects/0/einf914/data/20220508_1002_Plate07,33,'Agg','001P100N100C',20220502,True,True,True,True,True,...,True,"08.05.2022, 10:02:",20220508_1002_Plate07,7,'Carrot','Y','-',25,33_20220502,2022-05-08 10:02:00
5,/projects/0/einf914/data/20220503_0711_Plate07,33,'Agg','001P100N100C',20220502,True,True,True,True,True,...,True,"03.05.2022, 07:11:",20220503_0711_Plate07,7,'Carrot','Y','-',25,33_20220502,2022-05-03 07:11:00
7,/projects/0/einf914/data/20220624_0707_Plate01,84,'A5sp3','001P100N100C',20220621,True,True,True,True,True,...,True,"24.06.2022, 07:07:",20220624_0707_Plate01,1,'Carrot','Y','-',25,84_20220621,2022-06-24 07:07:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2129,/projects/0/einf914/data/20220627_1215_Plate01,84,'A5sp3','001P100N100C',20220621,True,True,True,True,True,...,True,"27.06.2022, 12:15:",20220627_1215_Plate01,1,'Carrot','Y','-',25,84_20220621,2022-06-27 12:15:00
2130,/projects/0/einf914/data/20220329_2114_Plate15,938,'A5','001P100N200C',20220325,True,True,True,True,True,...,True,"29.03.2022, 21:14:",20220329_2114_Plate15,15,'Carrot','Y','-',25,938_20220325,2022-03-29 21:14:00
2131,/projects/0/einf914/data/20220510_0959_Plate07,33,'Agg','001P100N100C',20220502,False,False,False,False,False,...,False,"10.05.2022, 09:59:",20220510_0959_Plate07,7,'Carrot','Y','-',25,33_20220502,2022-05-10 09:59:00
2132,/projects/0/einf914/data/20220507_1202_Plate07,33,'Agg','001P100N100C',20220502,True,True,True,True,True,...,True,"07.05.2022, 12:02:",20220507_1202_Plate07,7,'Carrot','Y','-',25,33_20220502,2022-05-07 12:02:00


In [ ]:
time = "2:00:00"
name_job = "malin"

list_args = [[]] * len(list_f)
overwrite = True
num_parallel = 6
run_parallel_post(
    "time_edge_post_process.py",
    list_f,
    list_args,
    [directory, overwrite],
    analysis_folders,
    num_parallel,
    time,
    "edge_post_process",
    cpus=32,
    name_job=name_job,
    node="fat",
)

In [28]:
import pandas as pd

# list_args= [[500],[1000],[2000]]+[[]]
# list_args= [[500]]
overwrite = True
load_graphs = True
num_parallel = 20
time = "5:00:00"
for index, row in analysis_folders.iterrows():
    folder = row["folder_analysis"]
    path_time_plate_info = row["path_time_plate_info"]
    plate = row["Plate"]
    num_cpus = 32
    if os.path.isfile(f"{directory_targ}{path_time_plate_info}"):
        whole_plate_info = pd.read_json(
            f"{directory_targ}{path_time_plate_info}", convert_dates=True
        ).transpose()
        whole_plate_info.index.name = "t"
        whole_plate_info.reset_index(inplace=True)
        run_parallel_post(
            "recompute_width.py",
            [],
            [],
            [directory_targ, overwrite, load_graphs],
            whole_plate_info,
            num_parallel,
            time,
            "recompute_width",
            cpus=num_cpus,
            name_job="recompute2",
            node="fat",
        )

sbatch: Single-node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: You will be charged for 0.25 node. A full node consists of 128 CPU cores, 983040 MiB of memory and 0 GPUs and can be shared by up to 4 jobs.


Submitted batch job 1944913


sbatch: Single-node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: You will be charged for 0.25 node. A full node consists of 128 CPU cores, 983040 MiB of memory and 0 GPUs and can be shared by up to 4 jobs.


Submitted batch job 1944914


sbatch: Single-node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: You will be charged for 0.25 node. A full node consists of 128 CPU cores, 983040 MiB of memory and 0 GPUs and can be shared by up to 4 jobs.


...

Submitted batch job 1944987


sbatch: Single-node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: You will be charged for 0.25 node. A full node consists of 128 CPU cores, 983040 MiB of memory and 0 GPUs and can be shared by up to 4 jobs.


Submitted batch job 1944988
Submitted batch job 1944989


sbatch: Single-node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: You will be charged for 0.25 node. A full node consists of 128 CPU cores, 983040 MiB of memory and 0 GPUs and can be shared by up to 4 jobs.
sbatch: Single-node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: You will be charged for 0.25 node. A full node consists of 128 CPU cores, 983040 MiB of memory and 0 GPUs and can be shared by up to 4 jobs.


Submitted batch job 1944990


In [ ]:
name_job = "recompute"

list_f = [
    get_tot_length_C,
    get_tot_length_straight,
    get_time_since_start,
    get_time_since_begin_exp,
    get_width_edge,
    get_pos_x,
    get_pos_y,
]
list_args = [{}] * len(list_f)
# list_args= [[500],[1000],[2000]]+[[]]
# list_args= [[500]]
overwrite = True
load_graphs = True
num_parallel = 32
time = "3:00:00"
for index, row in analysis_folders.iterrows():
    folder = row["folder_analysis"]
    path_time_plate_info = row["path_time_plate_info"]
    plate = row["Plate"]
    num_cpus = 32
    if os.path.isfile(f"{directory_targ}{path_time_plate_info}"):
        whole_plate_info = pd.read_json(
            f"{directory_targ}{path_time_plate_info}", convert_dates=True
        ).transpose()
        whole_plate_info.index.name = "t"
        whole_plate_info.reset_index(inplace=True)
        run_parallel_post(
            "time_edge_post_process.py",
            list_f,
            list_args,
            [directory_targ, overwrite, load_graphs],
            whole_plate_info,
            num_parallel,
            time,
            "edge_post_process",
            cpus=num_cpus,
            name_job=name_job,
            node="fat",
        )
        # break

In [10]:
folders = folders.sort_values(by="datetime")
folders

,total_path,Plate,PrincePos,root,strain,medium,split,Pbait,CrossDate,Temperature,...,/Analysis/transform_corrupt.mat,/Analysis/skeleton_realigned_compressed.mat,/Analysis/nx_graph_pruned.p,/Analysis/nx_graph_pruned_width.p,/Analysis/nx_graph_pruned_labeled.p,date,folder,Temp,unique_id,datetime
783,/projects/0/einf914/data/20220506_2350_Plate03,1048,3,'Carrot','C2','001P100N100C','Y','-',20220506,25,...,False,True,True,True,True,"06.05.2022, 23:50:",20220506_2350_Plate03,NaN,1048_20220506,2022-05-06 23:50:00
552,/projects/0/einf914/data/20220507_0150_Plate03,1048,3,'Carrot','C2','001P100N100C','Y','-',20220506,25,...,False,True,True,True,True,"07.05.2022, 01:50:",20220507_0150_Plate03,NaN,1048_20220506,2022-05-07 01:50:00
3050,/projects/0/einf914/data/20220507_0350_Plate03,1048,3,'Carrot','C2','001P100N100C','Y','-',20220506,25,...,False,True,True,True,True,"07.05.2022, 03:50:",20220507_0350_Plate03,NaN,1048_20220506,2022-05-07 03:50:00
3156,/projects/0/einf914/data/20220507_0550_Plate03,1048,3,'Carrot','C2','001P100N100C','Y','-',20220506,25,...,False,True,True,True,True,"07.05.2022, 05:50:",20220507_0550_Plate03,NaN,1048_20220506,2022-05-07 05:50:00
2150,/projects/0/einf914/data/20220507_0750_Plate03,1048,3,'Carrot','C2','001P100N100C','Y','-',20220506,25,...,False,True,True,True,True,"07.05.2022, 07:50:",20220507_0750_Plate03,NaN,1048_20220506,2022-05-07 07:50:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2323,/projects/0/einf914/data/20220619_1118_Plate30,1048,30,'Carrot','C2','001P100N100C','Y','-',20220506,25,...,False,True,True,True,True,"19.06.2022, 11:18:",20220619_1118_Plate30,NaN,1048_20220506,2022-06-19 11:18:00
827,/projects/0/einf914/data/20220619_1718_Plate30,1048,30,'Carrot','C2','001P100N100C','Y','-',20220506,25,...,False,True,True,True,True,"19.06.2022, 17:18:",20220619_1718_Plate30,NaN,1048_20220506,2022-06-19 17:18:00
939,/projects/0/einf914/data/20220619_2318_Plate30,1048,30,'Carrot','C2','001P100N100C','Y','-',20220506,25,...,False,True,True,True,True,"19.06.2022, 23:18:",20220619_2318_Plate30,NaN,1048_20220506,2022-06-19 23:18:00
1275,/projects/0/einf914/data/20220620_0518_Plate30,1048,30,'Carrot','C2','001P100N100C','Y','-',20220506,25,...,False,True,True,True,True,"20.06.2022, 05:18:",20220620_0518_Plate30,NaN,1048_20220506,2022-06-20 05:18:00


In [11]:
exp = Experiment(directory_targ)
exp.load(folders[:200:30])
exp.dates.sort()

/gpfs/home2/cbisot/pycode/AMFtrack/amftrack/pipeline/functions/image_processing/experiment_class_surf.py:104: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.folders["date"], format="%d.%m.%Y, %H:%M:"


2022-05-06 23:50:00
2022-05-09 15:39:00
2022-05-12 05:07:00
2022-05-14 17:52:00
2022-05-17 06:14:00
2022-05-20 10:38:00
2022-05-24 23:55:00


In [12]:
for t in range(exp.ts):
    exp.load_tile_information(t)

In [13]:
for t in range(exp.ts):
    exp.load_tile_information(t)

load_graphs(exp, directory_targ, post_process=True)

In [ ]:
t = 0
plot_edge_width(exp, t, lambda edge: edge.width(t), dilation=20)

In [15]:
import networkx as nx

resolution = 50
skip = False
for t in range(exp.ts)[:1]:
    (G, pos) = exp.nx_graph[t], exp.positions[t]
    edge_test = get_width_info_new(exp, t, resolution=resolution, skip=skip)
    nx.set_edge_attributes(G, edge_test, "width")
    exp.nx_graph[t] = G

True


In [16]:
save_graphs(exp)

2022-05-06 23:50:00 True
2022-05-09 15:39:00 True
2022-05-12 05:07:00 True
2022-05-14 17:52:00 True
2022-05-17 06:14:00 True
2022-05-20 10:38:00 True
2022-05-24 23:55:00 True


In [ ]:
t = 2
intervals = [[1, 3], [3, 4], [4, 5], [5, 6], [6, 7], [7, 9]]

plot_edge_width(exp, t, lambda edge: edge.width(t), dilation=20, intervals=intervals)